# Python API: FFlyRepo and FlightCollection Classes

This notebook demonstrates features of both the FFlyRepo and FlightCollection classes.

The FFlyRepo class represents one repository (archive) of FIREfly HDF5 files and their source Ch10 files.

The FlightCollection class represents zero or more FIREfly HDF5 files in a FIREfly repository that satisfy specific filtering criteria.

In [ ]:
from firefly import FFlyRepo, FlightCollection

## Specifying FIREfly Data Repository

A FIREfly repository is specified with s URI, which in this case can be as simple as a _directory path_.

In [ ]:
ffly_repo = FFlyRepo('/FIREfly/h5/', bucket='firefly-hsds')

Basic information about the FFlyRepo object:

In [ ]:
ffly_repo

## Various Methods of Data Filtering

### Filename Pattern

The pattern must be a valid Python regexp and is applied to FIREfly file names.

In [ ]:
fc = ffly_repo.filter(pattern='.*-2015')

The result of applying the filtering condition is a FlightCollection object:

In [ ]:
type(fc).__name__

Basic information about the FlightCollection object includes how many FIREfly files (flights) satisfied the filtering criteria:

In [ ]:
fc

To get just the number of flights in the collection:

In [ ]:
len(fc)

Flights in the collection can be selected using standard Python syntax. The statement below shows first ten FIREfly flights:

In [ ]:
fc[:10]

### Aircraft Information

Aircraft type:

In [ ]:
fc = ffly_repo.filter(aircraft='F-22')
fc

Listing last three FIREfly flights in the collection:

In [ ]:
fc[-3:]

Aircraft tail (serial) number:

In [ ]:
fc = ffly_repo.filter(tail='ED020020')
fc

Showing all FIREfly flights in the collection:

In [ ]:
fc[:]

### Parameter Values

FIREfly flights can be filtered based on values of the stored parameters. Filtering conditions are expressed as a set of parameter ranges each with a min and max value. Both open and closed ranges are supported. Python's `None` as either the min or max range value indicates left- and right-bounded range, respectively.

An open range is defined as a Python's tuple object: `(min_val, max_val)`; a closed interval is defined as a Python's list object: `[min_val, max_val]`.

#### Example 1

The filter condition below will select only the flights where aircraft altitude is greater than 5000 feet:

In [ ]:
fc = ffly_repo.filter(altitude=(5000, None))
fc

Every 200th file in the collection is listed below:

In [ ]:
fc[::200]

#### Example 2

It is possible to combine several parameters. In addition to altitude greater than 5000 feet, the aircraft speed must be greater or equal to 320 knots:

In [ ]:
fc = ffly_repo.filter(altitude=(5000, None), speed=[320, None])
fc

Filter expression for selecting flights is available:

In [ ]:
fc.flight_filter

#### Example 3

Any flight in June 2005:

In [ ]:
fc = ffly_repo.filter(time=('2005-06-01', '2005-07-01'))
fc

In [ ]:
fc[:]

## Accessing Filtered Flight Data

Filtered data for each FIREfly flight in the FlightCollection is available via an iterator. The filtering condition is applied iteratively to each flight and results in one or more FlightSegment objects with just the filtered data. Each of these FlightSegment objects will contain a **continuous** segment of one flight's data which means there could be more FlightSegment objects for the same flight in the collection.

An example below is a filtering condition using several different flight parameters and properties that will yield more than one FlightSegment for the same flight.

In [ ]:
fc = ffly_repo.filter(tail = ['ED020020', 'ED010085'],
                      time=('2012-01-01', '2013-01-01'),
                      altitude=[4000, None],
                      speed=[220, 300])
fc

In [ ]:
fc[:]

The filtering condition that will be applied to each flight's data is available from:

In [ ]:
fc.data_filter

Applying the filter condition is done via a Python generator which will return one FlightSegment object on each invocation:

In [ ]:
segs = fc.apply()
segs

In [ ]:
flight_seg1 = next(segs)
flight_seg1

In [ ]:
flight_seg2 = next(segs)
flight_seg2

In [ ]:
flight_seg3 = next(segs)
flight_seg3

In [ ]:
flight_seg4 = next(segs)
flight_seg4

In [ ]:
try:
    next(segs)
except StopIteration:
    print('Caught StopIteration. The generator has no new FlightSegment objects.')

In this case we ended up with four FlightSegment objects from the two flights in the FlightCollection.

The duration and data preview of the first FlightSegment:

In [ ]:
flight_seg1.duration

In [ ]:
flight_seg1.quickview('/derived/aircraft_ins')

The duration and data preview of the fourth FlightSegment:

In [ ]:
flight_seg4.duration

In [ ]:
flight_seg4.quickview('/derived/aircraft_ins')

## Accessing Flights in the FlightCollection

It is also possible to get to the flights in the FlightCollection without filtering their data. Again, it is a generator emitting one flight as a FlightSegment object at a time:

In [ ]:
for s in fc.flights:
    print(s)

These two FlightSegment objects contain entire flight's data.